# Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Import packages

In [1]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

### Q1 answer `19`

In [2]:
df_orig = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

len(df_orig.columns)


19

In [3]:
df_orig.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

### Q2 answer `42.59`

In [4]:
df_orig['duration'] = df_orig.tpep_dropoff_datetime - df_orig.tpep_pickup_datetime
df_orig.duration = df_orig.duration.apply(lambda td: td.total_seconds() / 60)

df_orig['duration'].std()

42.59435124195458

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

### Q3 answer `98%`

In [5]:
# before
print(f'Shape Before: {df_orig.shape}')

# drop outliers
df = df_orig[(df_orig.duration >= 1) & (df_orig.duration <= 60)]

# after
print(f'Shape After: {df.shape}')

print(f'fraction of the records left: {df.shape[0]/df_orig.shape[0]}')

Shape Before: (3066766, 20)
Shape After: (3009173, 20)
fraction of the records left: 0.9812202822125979


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

### Q4 answer `515`

In [6]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

/tmp/ipykernel_68499/2706819873.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [7]:
X_train.shape

(3009173, 515)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

### Q5 answer `7.64`

In [8]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261936284003

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

### Q6 answer `7.81`


In [9]:
target = 'duration'

df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

# drop outliers
df = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

df[categorical] = df[categorical].astype(str)

val_dicts = df[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

y_val = df[target].values

lr = LinearRegression()
lr.fit(X_val, y_val)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

/tmp/ipykernel_68499/645681582.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


7.7790391712905675

## Make functions

In [10]:
# global constants
categorical = ['PULocationID', 'DOLocationID']

target = ['duration']

def preprocess_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df[categorical] = df[categorical].astype(str)
    print(df.shape)
    
    return df

In [11]:
df_train = preprocess_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = preprocess_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

(3009173, 20)
(2855951, 20)


In [24]:
def train(df):

    # print(df.shape)
    # display(df)

    dv = DictVectorizer()

    y_train = df[target].values
    df = df.drop(columns=target)

    # print(df.shape)
    # print(y_train.shape)
    X_train = dv.fit_transform(df.to_dict(orient='records'))

    model = LinearRegression()
    model.fit(X_train, y_train)
    
    return (dv, model)


In [25]:
def predict(df, dv, model):

    # print(df.shape)
    y_val = df[target].values
    df = df.drop(columns=target)

    # print(df.shape)
    # print(y_val.shape)

    X_val = dv.transform(df.to_dict(orient='records'))

    model.fit(X_val, y_val)

    y_pred = model.predict(X_val)

    return root_mean_squared_error(y_val, y_pred)

In [26]:
dv, model = train(df_train[categorical + target])
rmse = predict(df_val[categorical + target], dv, model)
print(f'{rmse:.02f}')

(3009173, 3)
(3009173, 2)
(3009173, 1)
(2855951, 3)
(2855951, 2)
(2855951, 1)
7.78


In [27]:
print(f'{rmse}, {rmse:.4f}')

7.7790391712905675, 7.7790
